# Lesson 12.2 PyTorch深度学习建模可视化工具TensorBoard的安装与使用

&emsp;&emsp;在深度学习建模过程中，为了能够快速绘制模型基本结构、观察模型评估指标伴随训练过程的动态变化情况，当然也为了能够观察图像数据，我们可以使用TensorBoard工具来进行PyTorch深度学习模型的可视化展示。      
&emsp;&emsp;Tensorboard原本是TensorFlow的可视化工具，而目前在TensorboardX工具的加持下，其他深度学习计算框架也可使用TensorBoard工具进行可视化操作了。在PyTorch原生的可视化工具不够友好的情况下，我们更推荐使用TensorboardX来实现PyTorch的建模可视化。      
&emsp;&emsp;根据此前菜菜老师介绍的PyTorch整体架构图，虽然我们可以在torch.utils模块调用Tensorboard，但Tensorboard并不和PyTorch同步维护，要使用Tensorboard，需要单独安装。

![15](https://i.loli.net/2021/02/04/YaVEoCzJLrPNRjm.jpg)

> 值得一提的是，TensorBoard采用的是是本地开启服务、本地记录结果、Web读取结果的模式，和本地生成结果、在REPL环境中即时读取的模式有很大区别。

In [2]:
# 随机模块
import random

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt

# numpy
import numpy as np

# pytorch
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader

## 1.安装TensorboardX

&emsp;&emsp;和python安装第三方库一样，直接在命令行中安装即可。已经安装了TensorFlow的同学，在安装TensorFlow的过程中会自动安装Tensorboard，后续可以直接使用。当然，只安装了PyTorch的同学可根据以下步骤单独TensorboardX组件：

- Step 1.打开cmd命令行

可以从Anaconda Navigator进入cmd命令行

![16](https://i.loli.net/2021/02/04/EdSp1QuWVfj96mv.jpg)

或者直接win+R输入cmd进入命令行

<img src="https://i.loli.net/2021/02/04/IL15rMhsp3YmcUT.jpg" alt="17" style="zoom:50%;" />

当然，我们也可以直接在jupyterLab中打开命令行。（后续会经常使用jupyterLab命令行功能）      
首先，在jupyterLab中选择新建一个Luncher

<img src="https://i.loli.net/2021/02/04/efFztnOjymlHEo5.jpg" alt="21" style="zoom:50%;" />

然后选择新建一个Terminal

<img src="https://i.loli.net/2021/02/04/Noi2xP3Zw4Fpyjn.jpg" alt="19" style="zoom:40%;" />

即可打开一个命令行界面

- Step 2.使用pip进行安装

```shell
pip install tensorboardX
```

![20](https://i.loli.net/2021/02/04/GlFZEeXHJ1tR3vY.jpg)

Step 3.测试安装结果

导入相关的包，若能顺利导入，则证明安装成功

In [3]:
from torch.utils.tensorboard import SummaryWriter

## 2.SummaryWriter类与Tensorboard的基本使用

&emsp;&emsp;在进行建模过程可视化展示的过程中，核心使用的类就是SummaryWriter类，通过该类的使用，我们可以记录关键运算关系、模型数值指标等，进而可以进行展示。      
&emsp;&emsp;不过，考虑到真实生产环境中，大规模深度学习模型都是部署在服务器上运行的，我们无法像在本地IDE上一边输入数据一边绘图观测结果，我们只能将需要记录的关键结果记录下来并保存在某个文件里，然后再在本地或者服务器上开启某项服务读取该文件保存的信息。因此，整个过程会比在本地绘图稍微复杂，但确是真实生产环境中的主流操作。      
&emsp;&emsp;使用SummaryWriter进行记录并读取结果的一般过程如下：

![22](https://i.loli.net/2021/02/04/CcYrna16sF7842L.jpg)

> 服务器远程开启服务，本地通过Web服务来读取或者操作，也是在执行分布式计算过程中的常规操作。

- 实例化summarywriter

In [9]:
writer = SummaryWriter(log_dir='test')

实例化的过程中需要确定保存路径，当保存在主目录文件内时，只需输入文件夹名称即可。当文件夹不存在时，系统会自动创建一个。每次实例化一次SummaryWriter，就会在对应文件夹内新生成一个文件。单个文件夹内允许存在多个文件，在实际读取过程中会一次性读取文件夹内全部文件。

<img src="https://i.loli.net/2021/02/04/fZw7zn46BtKbRHc.jpg" alt="24" style="zoom:40%;" />

In [4]:
writer.log_dir       # 通过log_dir查看summary对象记录文件的位置

'test'

> 此外，在实例化SummaryWriter类的时候还有一个常用的关键参数——comment，通过comment参数的设置，我们可以自由设置每个实例化类的过程中文件名称的后缀。不过部分版本无法正常实现，同学们可以自行尝试。

- 记录数据

In [5]:
for i in range(10):
    writer.add_scalar('mul', i*i, i)

此处我们使用了add_scalar方法进行数据记录，该方法用于记录一组组标量，在实际记录过程中，通过迭代的方式逐个增加。其中，第一个参数代表改组记录的名称（也就是生成图像的名称），第二个参数代表y值，第三个参数代表x值。完成记录后，即可读取文件查看记录结果。除此以外，常用的还有add_graph方法用于记录深度学习建模流程。

- 启动服务读取文件

&emsp;&emsp;接下来，在本地启动服务，读取刚刚生成的记录文件。同样，我们可以在jupyter中的命令行直接启动服务。

```shell
F:
cd "F:\Code File\PyTorch实战\"
tensorboard --logdir="test"
```

> 注意，如果不是通过jupyter，而是通过cmd打开的命令行，如果要进入其他盘的主目录，例如F盘，则需要先输入F:指令，将当前操作主目录转移到F盘上。

<img src="https://i.loli.net/2021/02/04/1t7NqxMEp3KbP54.jpg" alt="26" style="zoom:40%;" />

命令行中内容，其中tensorboard为指令关键词，`--logdir`参数输入记录文件夹地址（注意是文件夹地址而不是文件夹内的文件地址）。在默认情况下，系统会自动启动6006端口，可用于Web调用服务，这种调用方法类似于jupyter服务也是通过Web调用的，jupyter端口默认是8888。需要注意的是，上述命令中，是先将当前操作目录转移至test文件目录下，然后在调用tensorboard命令时只需要在--logdir参数位输入文件名称即可。无论是在--logdir参数位输入完整的文件路径，还是先将当前操作目录转移到对应目录下再直接输入文件名，都可以直接调用服务。

> 命令行的操作是需要区分当前操作目录的，命令行中光标前面的内容即是当前操作目录，每次打开命令行时默认主目录都是当前用户的主目录，我们可以通过输入dir(Linux使用ls或者ll命令）查看当前文件内容        
<img src="https://i.loli.net/2021/02/04/6A2bfROGZ89sdXF.jpg" alt="27" style="zoom:40%;" />      
使用cd命令则是移动当前主目录，例如，下图中移动到F盘的Code File文件夹内的PyTorch实战文件夹，再使用dir命令查看当前文件      
<img src="https://i.loli.net/2021/02/04/SX9tjVLo3qRgDbC.jpg" alt="28" style="zoom:40%;" />   

- 进入6006端口页面

&emsp;&emsp;打开浏览器，在地址栏输入localhost:6006即可进入到tensorboard展示页面。由于此前我们已经通过write记录了一组数据，我们可以看到展示效果。

![30](https://i.loli.net/2021/02/04/E6PDTmJuwhHpMfK.jpg)

接下来即可对图形进行简单操作，最基本的，我们可以按住alt键，滚动鼠标滚轮对其进行缩放，以及通过鼠标左键对其进行拖拽。更多功能我们将在后续实际使用过程中逐步介绍。

另外，如果默认6006端口冲突，也可自行修改服务端口号，只需在启动命令后后输入参数--port XXXX即可，例如，通过如下方法修改服务端口号至7007。当然，通过Web调用的服务，也可以远程调用，类似jupyter的远程调用，只需要对其配置文件进行修改即可。

<img src="https://i.loli.net/2021/02/04/8cd4mlSrfT9PsiW.jpg" alt="29" style="zoom:40%;" /> 

![31](https://i.loli.net/2021/02/04/2qulZsrdJkPR7yw.jpg)

&emsp;&emsp;至此，简单的跑通一个使用Tensorboard的使用实验顺利完成，在后续的建模过程中，我们还将进一步熟悉Tensorboard的其他功能。值得一提的是，尽管目前看起来是Tensorboard只记录了一张折线图，但实际上add_scalar方法核心应用场景是用于记录模型运行过程中，随着迭代次数增加一些数值指标的变化情况，如loss值、准确率值等等等等。无论如何，我们需要知道，Tensorboard的核心应用场景是深度学习建模过程的可视化展示，若只用其进行绘图，则有些大材小用了（并且有把简单问题复杂化的嫌疑）。